In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sike
import numpy as np
import sike.analysis.post_processing as spp
import sike.analysis.plotting as sp
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
# sike.setup(elements = ["H"], savedir = "..")

In [ ]:
num_x = 10
Te = np.linspace(15.0,30.0,num_x)
ne = 1e20*np.ones(num_x)
el = "Ne"
# vgrid = np.sqrt(2.0 * sike.utils.constants.EL_CHARGE * np.geomspace(2.5e-4,30e3,500) / sike.utils.constants.EL_MASS)
c = sike.SIKERun(Te=Te, 
                 ne=ne, 
                 element=el, 
                 resolve_j = False, 
                 resolve_l = False, 
                 saha_boltzmann_init=False, 
                 ionization= True,
                 excitation= True,
                 emission= True,
                 autoionization= True,
                 radiative_recombination= True,
                #  vgrid = vgrid
                 )


In [ ]:
# ds = c.evolve(1e6)
ds = c.solve()

In [63]:
# sp.plot_nz(ds,logy=False, logx=False, normalise=False)
sp.plot_Zavg(ds)

In [53]:
iz_trans = [t for t in c.impurity.transitions if t.type == "ionization"]
rr_trans = [t for t in c.impurity.transitions if t.type == "radiative_recombination"]

In [ ]:
rr_trans[-1].__dict__

In [51]:
fig,ax = plt.subplots(1)
for t in rr_trans[-1:]:
    ax.plot(t.sigma)
ax.set_yscale("log")

In [ ]:
sum([t.delta_E.sum() for t in iz_trans])

In [ ]:
iz_trans[0].__dict__

In [19]:
M = ds.M.isel(x=0).values * c.t_norm
# np.savetxt("M_new.csv",M_new)
M_old = np.loadtxt("/Users/power8/Documents/05_papers/01_impurities/01_analysis/M_old.csv")

In [ ]:
M[:,0] - M_old[:,0]

In [ ]:
import sike.utils

num_x = 10
Te = np.linspace(1.0,10.0,num_x)
ne = 1e20*np.ones(num_x)
el = "Ne"

T_hot = 100 * np.ones(num_x)
n_hot_frac = 0.01
vgrid = np.geomspace(min(sike.utils.constants.EXTENDED_VGRID),max(sike.utils.constants.EXTENDED_VGRID),500)
fe_bimax = sike.analysis.plasma_utils.get_bimaxwellians(n1=(1.0-n_hot_frac)*ne, T1=Te, n2=n_hot_frac*ne,T2=T_hot, vgrid=vgrid)

c_hf = sike.SIKERun(fe=fe_bimax, vgrid=vgrid, element=el, resolve_j = False, resolve_l = False, saha_boltzmann_init=True)
c_nhf = sike.SIKERun(Te=Te, 
                     ne=ne, 
                     element=el, 
                     resolve_j = False, 
                     resolve_l = False, 
                     saha_boltzmann_init=True)

In [ ]:
# ds_hf = c_hf.evolve(1e5)
# ds_nhf = c_nhf.evolve(1e5)
ds_hf = c_hf.solve()
ds_nhf = c_nhf.solve()

In [60]:
# sp.plot_nz(ds_hf,logy=False, logx=True, normalise=False)
# sp.plot_nz(ds_nhf,logy=False, logx=True, normalise=False)
sp.plot_Zavg(ds_hf)
sp.plot_Zavg(ds_nhf)